## Example how the chessboard distance is computed

This notebook show the workflow of the chessboard-distance metric for two different example data sets of (Intel D405 and ZED-min).

In [ ]:
# load submodule in parent directory
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
# load dependencies
import cv2
import open3d as o3d

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

from inout import bag_reader,svo_reader

mpl.rcParams['figure.dpi']= 100

Extract the images(.png) and 3D data as point-cloud(.ply) from the camera recording files, such as .bag for intel cameras and .svo for stereolabs cameras.

For the example the files are extracted and saved.

In [ ]:
# bag-file 
data_path = "DATA_DIR_PATH" # adjust to the directory to the your data-path
file_name = "FILE_NAME.bag" # adjust to the file name you want to load
distance_mm = 160 # adjust ground truth distance
depth_list,image_list = bag_reader.extract_frames(os.path.join(data_path,file_name),None,1,distance_mm)

In [ ]:
# checker board settings
chessboard_shape = (8,10) # (height, width)
square_size = 7 # in mm

Load image and detect chessboard corners

In [ ]:
image = image_list[0]
plt.imshow(image)

In [ ]:
image_grey = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
ret, corners = cv2.findChessboardCorners(image_grey, chessboard_shape, None)
corners = corners.squeeze()
image_grey.shape

In [ ]:
i = 0
# corners = corners[::-1]
if ret:
    draw_points = cv2.drawChessboardCorners(image.copy(),chessboard_shape,corners,ret)
    for corner in corners:
        pos = (int(corner[0]),int(corner[1]))
        draw_points = cv2.putText(draw_points,str(i),pos,0,0.5,(0,0,0))
        i += 1
    plt.imshow(draw_points)
    print(corners.shape)

## Load 3D representation

In [ ]:
pcd = o3d.geometry.PointCloud()

depth = depth_list[0]
points = np.nan_to_num(depth).reshape(-1,3)
pcd.points = o3d.utility.Vector3dVector(points)

o3d.visualization.draw_geometries([pcd])

In [ ]:
point_3d = list()
point_2d = list()

if ret:
    for corner in corners:
        pos = (int(corner[1]),int(corner[0]))
        point_2d.append(pos)

        if depth[pos].all() != 0.:
            point_3d.append(depth[pos])
        else:
            point_3d.append(np.array([0.,0.,0.]))
            

# save original point
org_point_2d = np.array(point_2d)
org_point_3d = np.array(point_3d)
# reshape to chessboard size
point_3d = np.array(point_3d).reshape((chessboard_shape[1],chessboard_shape[0],3))
point_2d = np.array(point_2d).reshape((chessboard_shape[1],chessboard_shape[0],2))

# show example point here to see the right scaling and
# check it the resize end in the same point as original points
print(org_point_2d[1],org_point_3d[1])
point_2d[0][1],point_3d[0][1]

In [ ]:
pcd = o3d.geometry.PointCloud()

# show the selected/valid points of the chessboard in 3D
pcd.points = o3d.utility.Vector3dVector(org_point_3d)
o3d.visualization.draw_geometries([pcd])

In [ ]:
test_image = image.copy()

distance_list = list()
draw_c = 0

for i in range(len(point_3d)):
  for j in range(len(point_3d[0])-1):
     point_a = point_3d[i][j]
     point_b = point_3d[i][j+1]

     if point_a.all() != 0. and point_b.all() != 0.:
        dist = np.linalg.norm(point_a-point_b)
        distance_list.append(abs(dist-square_size))

        img_post_01 = (int(point_2d[i][j][1]),int(point_2d[i][j][0]))
        img_post_02 = (int(point_2d[i][j+1][1]),int(point_2d[i][j+1][0]))

        test_image = cv2.line(test_image,img_post_01,img_post_02,(69, 252, 3),2)

        draw_c += 2


for j in range(len(point_3d[0])):
   for i in range(len(point_3d)-1):
       point_a = point_3d[i][j]
       point_b = point_3d[i+1][j]
       
       if point_a.all() != 0. and point_b.all() != 0.:
          dist = np.linalg.norm(point_a-point_b)
          distance_list.append(abs(dist-square_size))
          img_post_01 = (int(point_2d[i][j][1]),int(point_2d[i][j][0]))
          img_post_02 = (int(point_2d[i+1][j][1]),int(point_2d[i+1][j][0]))

          test_image = cv2.line(test_image,img_post_01,img_post_02,(252, 165, 3),2)
          draw_c += 2

distance = np.array(distance_list)
_ = plt.imshow(test_image,)

In [ ]:
chess_distance = np.array(distance_list) # in mm
pd.DataFrame(chess_distance,columns=['Checker_distance']).describe()